In [41]:
import json
import requests
import pandas as pd
import datetime
import pytz

In [57]:
# From openuv 
api_key = "openuv-7orcjfrlo62ivj9-io"

# headers for api call
headers = {
    "x-access-token": api_key,
    "Content-Type": "application/json"
}

lat = 34.11
lng = -117.70
alt = 100
# Create url with api key above

url = "https://api.openuv.io/api/v1/forecast?lat={}&lng={}&alt={}&dt=2023-10-26T00:00:00-05:00".format(lat,lng,alt)

request = requests.get(url= url, headers=headers)
data = request.json()
df = pd.json_normalize(data['result'])
df = df.drop('sun_position.azimuth', axis = 1)
df = df.drop('sun_position.altitude', axis = 1)
df['uv_time'] = pd.to_datetime(df['uv_time'], format='ISO8601')
df['Time'] = pd.to_datetime(df['uv_time'])
df['Time'].dt.tz_convert(pytz.utc)
df['Time'].dt.tz_convert(pytz.timezone('US/Pacific')).head()
df = df.drop('uv_time', axis = 1)
df['Time'] = df['Time'].dt.strftime('%H:%M')
print(df)

KeyError: 'result'